In [ ]:
import os
import json

#Loading Math Vista Dataset
from datasets import load_dataset
from openai import OpenAI # type: ignore

os.environ["OPENAI_API_KEY"] = ""
dataset = load_dataset("AI4Math/MathVista")
client = OpenAI()

/home/local/ASURITE/skumbha4/.conda/envs/shri_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import json
from tqdm import tqdm
# Path to your JSON file
file_path = "./paraphrased_no_cot_no_image.json"

# Open and read the JSON file
with open(file_path, "r") as file:
    data = json.load(file)  # Parse JSON content into a Python dictionary

# Print the loaded JSON data
print(str(data[0]))

{'pid': '2', 'Ground Truth': '1000', 'Prediction': 'The total volume of the measuring cup is 500 grams.'}


In [3]:
evaulation = []
for sample in tqdm(data):
    chat_completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
        "role": "system",
        "content": 'You are a strict evaluator comparing a "Ground Truth" value and a "Prediction" value. Your task is to determine if they are equivalent. The values may be integers, integers written in words, or sentences where a numeric answer is embedded. If the Prediction matches the Ground Truth in meaning, return "True". Otherwise, return "False". Only respond with "True" or "False" without any additional explanation.'
        },
        {
        "role": "user",
        "content": str(sample)
        }
    ]
    )

    response_text = chat_completion.choices[0].message.content

    sample['eval'] = response_text
    evaulation.append(sample)

    # print(response_text)
# File name
file_name = "evaulations/eval_paraphrased_no_cot_no_image.json"

# Writing the list to a JSON file
with open(file_name, "w") as json_file:
    json.dump(evaulation, json_file, indent=4)  # 'indent=4' makes the JSON file more readable

print(f"Written to {file_name}")

100%|██████████| 418/418 [03:31<00:00,  1.98it/s]

Written to evaulations/eval_paraphrased_no_cot_no_image.json


In [4]:
# Sample JSON data (replace this with your JSON data)

# Calculate accuracy
def calculate_accuracy(data):
    total_records = len(data)
    true_count = sum(1 for record in data if record["eval"] == "True")
    accuracy = (true_count / total_records) * 100 if total_records > 0 else 0
    return accuracy

# Calculate and print the accuracy
accuracy = calculate_accuracy(evaulation)
print(f"Accuracy: {accuracy:.2f}%")


Accuracy: 8.13%


In [5]:
len(data)

418